<a href="https://colab.research.google.com/github/Daniel-BTCSH/Text-to-Video/blob/main/speechface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#判断人脸
!pip install face_recognition

In [ ]:
!pip install paddlepaddle-gpu==2.4.1.post116 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
import paddle.fluid

In [ ]:
# 语音
!pip install pytest-runner 
!pip install paddlespeech==1.4.0


In [ ]:
#@title
import base64
import librosa
import soundfile as sf

def resample_rate(path,new_sample_rate = 16000):
    signal, sr = librosa.load(path, sr=None)
    wavfile = path.split('/')[-1]
    wavfile = wavfile.split('.')[0]
    file_name = wavfile + '_new.wav'
    new_signal = librosa.resample(signal, sr, new_sample_rate) # 
    #librosa.output.write_wav(file_name, new_signal , new_sample_rate) 
    sf.write(file_name, new_signal, new_sample_rate, subtype='PCM_24')
    print(f'{file_name} has download.')
    return file_name

In [ ]:
!python --version

In [ ]:
!pip install ppgan
!pip uninstall typeguard
!pip install typeguard==2.13.3


In [ ]:
from ppgan.apps.wav2lip_predictor import Wav2LipPredictor
from paddlespeech.cli.asr.infer import ASRExecutor
from paddlespeech.cli.tts.infer import TTSExecutor
from paddlenlp import Taskflow
from ppgan.apps.wav2lip_predictor import Wav2LipPredictor
from ppgan.apps.first_order_predictor import FirstOrderPredictor
from ppdiffusers import StableDiffusionPipeline

In [ ]:

# 可选模型权重
# CompVis/stable-diffusion-v1-4
# runwayml/stable-diffusion-v1-5
# stabilityai/stable-diffusion-2-base （原始策略 512x512）
# stabilityai/stable-diffusion-2 （v-objective 768x768）
# Linaqruf/anything-v3.0
# ......


# from paddlenlp import Taskflow
# text_to_image = Taskflow("text_to_image")

# # https://github.com/JiehangXie/PaddleBoBo/blob/0.1/PaddleTools/GAN.py
sd = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
asr = ASRExecutor()
tts = TTSExecutor()
dialogue = Taskflow("dialogue")
wav2lip_predictor = Wav2LipPredictor(face_det_batch_size = 2,wav2lip_batch_size = 16,face_enhancement = True)

import base64,os
import librosa
import soundfile as sf

import hashlib

def get_prompt_id(text):
  m = hashlib.md5()
  m.update(text.encode("utf8"))
  return m.hexdigest()


def resample_rate(path,new_sample_rate = 16000):
    signal, sr = librosa.load(path, sr=None)
    wavfile = path.split('/')[-1]
    wavfile = wavfile.split('.')[0]
    file_name = wavfile + '_new.wav'
    new_signal = librosa.resample(signal, sr, new_sample_rate) # 
    #librosa.output.write_wav(file_name, new_signal , new_sample_rate) 
    sf.write(file_name, new_signal, new_sample_rate, subtype='PCM_24')
    print(f'{file_name} has download.')
    return file_name

# def text2img(text):
#   sd = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5",requires_safety_checker=False)
#   prompt = "a portrait of "+text+".passport photo， intricate. lifelike. soft light. sony a 7 r iv 5 5 mm. cinematic post - processing "
#   image = sd(prompt, 
#         guidance_scale=7.5,
#         num_inference_steps=21,
#         height=416, width=384).images[0]
#   sd=None
#   return {
#       "image":image,
#       "prompt":prompt,
#       "id":get_prompt_id(prompt)
#   }


def audio2text(input_file):
  input_file=resample_rate(input_file,new_sample_rate = 16000)
  result = asr(audio_file=input_file,device='cpu')
  print('audio2text')
  return result

def text2audio(text):
  out_file =tts(text,device='cpu')
  print('text2audio')
  return out_file

def reply(t):
  data = [t]
  result = dialogue(data)
  print('reply',result[0])
  return result[0]

def wav2lip(input_video,input_audio):
  out_file='video.mp4'
  wav2lip_predictor.run(input_video, input_audio, out_file)
  return out_file


def FOM(source_image,driving_video,output_path):
  output,filename = os.path.split(output_path)
  first_order_predictor = FirstOrderPredictor(output = output,filename = filename, 
                              face_enhancement = True, 
                              ratio = 0.4,
                              relative = True,
                              image_size=512,
                              adapt_scale = True)
  first_order_predictor.run(source_image, driving_video)
  return os.path.join(output,filename)

def write_wav(data, samplerate,wav_file):
  # data, samplerate = sf.read('existing_file.wav')
  sf.write(wav_file, data, samplerate)
  return wav_file

#判断人脸
import face_recognition

def check_face_image(image_path):
  image = face_recognition.load_image_file(image_path)
  face_locations = face_recognition.face_locations(image)
  print("I found {} face(s) in this photograph.".format(len(face_locations)))

  if len(face_locations)==1:
    # top, right, bottom, left = face_locations[0]
    # width=right-left
    # height=bottom-top
    # s=(width*height)/(image.width*image.height)
    # if s>0.4:
    return True
  return False


# from fastapi import FastAPI, APIRouter,Body
# from fastapi.middleware.cors import CORSMiddleware
# from fastapi.responses import HTMLResponse

# app = FastAPI()
# api_router = APIRouter()

# origins = ['*']

# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=origins,
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )



In [ ]:
!pip install gradio==3.22.0
import os
import gradio as gr

from google.colab import drive

# 挂载网盘
drive.mount('/content/drive/')  
# 切换路径
os.chdir('/content/drive/MyDrive/data')

portrait_video=None

def create_avatar(portrait_file,file_type):
  print('file_type::',file_type)
  url=''
  # avatar=text2img(text)
  #存下来，并索引
  # portrait_file=avatar['id']+'.png'
  # avatar['image'].save(portrait_file)
  # if check_face_image(portrait_file)==False:
  #     return portrait_file
  portrait_video=FOM(portrait_file,'driving_video.mp4','./portrait_video.mp4')
  if file_type=='gif':
    portrait_video=convert_mp4_to_gif(portrait_video)
    url='data:image/gif;base64,'+encode_base64(portrait_video)
  
  return '<img src="'+url+'" />'


def test(text,wav_file_input,input_type):
  portrait_video='./portrait_video.mp4'
  if wav_file_input!=None:
    wav_file=write_wav(wav_file_input[1],wav_file_input[0],'./wav_file.wav')
    text=audio2text(wav_file)
  
  q=reply(text)
  input_audio=text2audio(q)
  result=wav2lip(portrait_video,input_audio)
  return result

def test2(text,wav_file_input,input_type):
  portrait_video='./portrait_video.mp4'
  if input_type=='wav':
    input_audio=wav_file_input
  elif input_type=='text':
    input_audio=text2audio(text)
  
  result=wav2lip(portrait_video,input_audio)
  return result


def encode_base64(file):
    with open(file, 'rb') as f:
        img_data = f.read()
        base64_data = base64.b64encode(img_data)
        print(type(base64_data))
        # print(base64_data)
        # 如果想要在浏览器上访问base64格式图片，需要在前面加上：data:image/jpeg;base64,
        base64_str = str(base64_data, 'utf-8')
        # print(base64_str)
        return base64_str
    

import cv2
from PIL import Image
def convert_mp4_to_gif(input_file,step=None,duration=None):
    '''
    传参 :  input_file 视频文件名
            output_file gif文件名
            duration 每帧图像的停留时间 毫秒ms 
            step 跳帧,降低采样率,减小gif体积
    返回 : 无
    '''
    output,filename = os.path.split(input_file)
    video_capture = cv2.VideoCapture(input_file)
    rate=video_capture.get(5)
    still_reading, image = video_capture.read()
    frame_count = 0
    i = 0
    frames = []

    if duration==None:
      duration=1/rate

    #单位是秒
    print('rate',rate)
    if step==None:
      if rate>12:
        step=rate-12
        duration=step/rate
      else:
        step=1
    
        
    while still_reading:
        still_reading, image = video_capture.read()
        if not still_reading:
            break
        if i>step:
            frames.append(Image.fromarray(cv2.cvtColor(image.copy(),cv2.COLOR_BGR2RGB)))
            frame_count += 1
            i=0
        i+=1
    
    # v_duration=frame_count/rate
    

    frame_one = frames[0]
    output_file=os.path.join(output,'portrait.gif')
    frame_one.save(output_file, format="GIF", append_images=frames[1:],save_all=True, duration=duration, loop=0)
    return output_file





with gr.Blocks() as demo:
    with gr.Column():
        # avatar
        # input_text=gr.Textbox()
        input_i=gr.Image(type='filepath')
        input_file_type=gr.Radio(["mp4", "gif"],value='mp4', label="文件类型")
        # output_video1=gr.Video(format='mp4')
        # output_gif=gr.Image(type="numpy")
        
        btn = gr.Button(value="创建形象")
        btn.click(create_avatar, inputs=[input_i,input_file_type], outputs=[gr.HTML()],api_name='create_avatar')
    with gr.Column():
        # talking
        input_file_type=gr.Radio(["wav", "text"],value='wav', label="输入类型")
        input_audio=gr.Audio(label="录音",type="numpy",source='microphone')
        input_talk_text=gr.Textbox()
        output_video2=gr.Video()
        btn2 = gr.Button(value="生成视频")
        btn2.click(test2, inputs=[input_talk_text,input_audio,input_file_type], outputs=[output_video2],api_name='create_talk_face')
      
    # gr.Interface(fn=create_avatar, inputs=[create_avatar], outputs=[output_video1],
    #       layout="vertical")
    # gr.Interface(fn=test, inputs=[input_audio,input_dropdown], outputs=[output_video2],
    #       layout="vertical")

    demo.queue(concurrency_count=2)

    demo.launch(
                # server_name='0.0.0.0',
                share=True,
                debug=True,
                max_threads=2,
                show_error=True,
                show_api=True
                # file_directories=[root_path]
                )